In [1]:
import pickle
from collections import defaultdict

# 测试defaultdict是否能被pickle
dd = defaultdict(int)
dd[('a', 'b')] = 10

try:
    pickled = pickle.dumps(dd)
    print("defaultdict can be pickled")
except Exception as e:
    print(f"defaultdict cannot be pickled: {e}")


defaultdict can be pickled


In [ ]:
chr(29275)

In [ ]:
type("hello world".encode("utf8"))

In [ ]:
len("你好 世界".encode("utf8")), len("你好 世界")

```chr(0)``` 返回'\x00'

In [ ]:
chr(0)

打印 chr(0) 不可见

In [ ]:
print(chr(0))

In [ ]:
string = "this is a test" + chr(0) + "string"
string

In [ ]:
print(string)

In [ ]:
test_str = "hello 跃洋"
test_str_utf8 = test_str.encode("utf8")
test_str_utf8

In [ ]:
list(test_str_utf8)

In [ ]:
test_str_utf8.decode("utf-8")

In [ ]:
"".join([bytes([b]).decode("utf-8") for b in "hello world".encode("utf8")])

In [ ]:
import mmap

with open("/Users/yueyangcheng/WorkSpace/cs336/lab1/data/TinyStoriesV2-GPT4-valid.txt", mode="r+", encoding="utf-8") as f:
    with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_WRITE) as m:
        data = m.read(10)
        print(data)

In [ ]:
from typing import List, Tuple, Dict
from collections import defaultdict
import mmap
import random

def load_and_sample_data(file_path: str,
                         sample_size: int,
                         special_tokens: str = "<|endoftext|>"
                         ) -> str:
    try:
        with open(file_path, "r+", encoding="utf-8") as f:
            with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
                documents = []
                start = 0
                while start < len(mm):
                    end = mm.find(special_tokens.encode("utf-8"), start)
                    if end == -1:
                        end = len(mm)
                        doc = mm[start:].decode("utf-8").strip()
                        if doc:
                            documents.append(doc)
                        break;
                    doc = mm[start: end].decode("utf-8").strip()
                    if doc:
                        documents.append(doc)

                    start = end + len(special_tokens)
                if len(documents) > sample_size:
                    documents = random.sample(documents, sample_size)

                return special_tokens.join(documents)
    except Exception as e:
        raise IOError(f"Error opening {file_path}: {e}")

In [ ]:
load_and_sample_data("/Users/yueyangcheng/WorkSpace/cs336/lab1/data/TinyStoriesV2-GPT4-valid.txt", sample_size=10)

In [ ]:
test_list = ["hello", "world", "cyy"]
"".join(test_list)

In [ ]:
list("".join(test_list))

In [ ]:
import regex

GPT2_PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

def gpt_bytes_to_unicode_local() -> Dict[int, str]:
    bs = list(range(33, 127)) + list(range(161, 173)) + list(range(174, 256))
    cs = bs[:]

    n = 0
    for i in range(256):
        if i not in bs:
            bs.append(i)
            cs.append(n + 256)
            n += 1
    return {b: chr(c) for b, c in zip(bs, cs)}

def pre_tokenizer_document(doc: str, bytes_to_unicode_map: Dict[int, str]) -> List[List[str]]:
    tokens = regex.findall(GPT2_PAT, doc, flags=regex.UNICODE)
    sequences = []
    for token in tokens:
        token_unicode = "".join([bytes_to_unicode_map[b] for b in token.encode("utf-8")])
        sequences.append(list(token_unicode))

    return sequences

bytes_to_unicode_map = gpt_bytes_to_unicode_local()

pre_tokenizer_document("hello, world cyyang", bytes_to_unicode_map)

In [ ]:
regex.findall(GPT2_PAT, "hello world , cyyang dafda")

In [ ]:
from tqdm import tqdm
import time

# 示例：一个简单的循环
for i in tqdm(range(100)):
    time.sleep(0.1)  # 模拟耗时操作